>>>>>>>>> ## *BUSCANDO EL VASO*



>>>>>>>> <img src='https://raw.githubusercontent.com/cmosquer/softmax/master/vaso.jpg' alt="Drawing" style="width:5%" size=10/>

##Segmentación del vaso en tomografía computada abdominal

El objetivo de esta práctica es entrenar un modelo que segmente el área de imagen perteneciente al vaso en un corte de tomografía computada de abdomen. Para eso, utilizaremos un dataset de 41 tomografías donde se ha delimitado manualmente el área del vaso. 

<img src='https://raw.githubusercontent.com/cmosquer/softmax/master/spleendiagram.png' alt="Drawing" style="width:5%" size=10/>



Entra al siguiente link y, sobre el nombre de la carpeta, realizá "Agregar a Mi Unidad":

https://drive.google.com/open?id=1sXNLvCFVnSEkiU4DOB0j4Pei-NzgvOTT


In [0]:
import os

from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)


Verificá que en el panel de la izquierda, en la pestaña Archivos, el directorio incluya la carpeta gdrive. 

## Nifti

Las imágenes nifti (.nii o .nii.gz) son un tipo de dato diseñado para imágenes médicas que contiene información sobre el sampleo y otros parámetros de adquisición de la imagen.

El paquete nibabel es de los más usados para trabajar con imágenes nifti.

In [0]:
working_dir = '/content/gdrive/My Drive/Images/spleen_training/'  #Modificá el path a donde tenés en Mi Unidad la carptea spleen_training
images_path = os.path.join(working_dir,'images3D')
example_img_path = os.path.join(images_path,'spleen_2.nii.gz')


In [0]:
import nibabel as nib
import numpy as np

img = nib.load(example_img_path)
img

In [0]:
print(img)

In [0]:
def loadnifty(path,dim=None):

  img = nib.load(path)
  img_data = img.get_fdata() 
  return img_data
  

In [0]:

example_img = loadnifty(example_img_path)
type(example_img)

¿Cuántos cortes axiales tiene la tomografía del paciente 2? ¿Y la del paciente 10?

In [0]:
[...te toca implementar]

Graficá un corte axial. Graficá el histograma de la imagen con la función hist() de matplotlib

In [0]:
[...te toca implementar]

from matplotlib import pyplot as plt
f = plt.figure(figsize=(20,20))
# Visualizamos la imagen y el gt
plt.subplot(1,2,1)
plt.grid(False)
plt.imshow(....., cmap='gray')


plt.subplot(1,2,2)
plt.grid(False)
plt.title("Histograma de intesidades de la imagen")
_ = plt.hist(.....)

**Graficando las máscaras**


In [0]:
masks_path = os.path.join(working_dir,'labels3D')
example_mask_path = os.path.join(masks_path,'spleen_2.nii.gz')
example_img_path = os.path.join(images_path,'spleen_2.nii.gz')
example_img = loadnifty(example_img_path)
example_mask = loadnifty(example_mask_path)

print('Dimensión imagen', example_img.shape)
print('Dimensión máscara', example_mask.shape)

Graficá el corte 80 del paciente 2 y su correspondiente máscara.

In [0]:
from matplotlib import pyplot as plt

f = plt.figure(figsize=(15,15))

plt.subplot(1,3,1)
plt.grid(False)
plt.imshow(example_img[:,:,80], cmap='gray')


plt.subplot(1,3,2)
plt.grid(False)
plt.imshow(example_mask[:,:,80], cmap='jet')

plt.subplot(1,3,3)
plt.grid(False)
plt.imshow(example_img[:,:,80], cmap='gray')
plt.imshow(example_mask[:,:,80], cmap='jet', alpha = 0.5)

## Trabajando como imágenes 2D

En un primer enfoque, vamos a considerar cada corte axial como una imagen independiente. Vamos a utilizar solamente aquellos cortes que contengan algún área del vaso. Estos se encuentran en la carpeta 'images2D'


In [0]:
images_2D= os.path.join(working_dir,'images2D/')
labels_2D=os.path.join(working_dir,'labels2D')

**Recordamos el concepto de GENERADORES**

¿Que?: una función que devuelve un objeto (iterador) sobre el cual se puede iterar (un valor a la vez).
¿Como?: Definir una función normal con sentencia 'yield' en lugar de sentencia 'return'.

Si una función contiene por lo menos una sentencia 'yield', se convierte en generador. Puede tener múltiples yields y returns.
La diferencia es que una sentencia return termina por completo la función, mientras que una sentencia yield pone pausa a la función, guardando todos los estados de variables, y luego continua desde ese estado en llamados sucesivos.
<img src='https://raw.githubusercontent.com/cmosquer/softmax/master/iterable-vs-iterator.png' alt="Drawing" style="width:5%" size=10/>

In [0]:
#CON DISTINTOS YIELD
def my_gen():
    n = 1
    print('This is printed first')
    # Generator function contains yield statements
    yield n

    n += 1
    print('This is printed second')
    yield n

    n += 1
    print('This is printed at last')
    yield n

In [0]:
a = my_gen()

In [0]:
x = next(a)
print(x)

In [0]:
#CON CICLO
def read_str(my_str):
    length = len(my_str)
    for i in range(length):
        yield my_str[i]

In [0]:
str_gen = read_str('Aguante la pastafrola')

In [0]:
#Puedo iterar sobre mi objeto de generador pidiendo el siguiente llamado con next(), o iterarlo con for  y recorrer todos sus llamados

print(next(str_gen))

In [0]:
for i in str_gen:
  print(i)

## Generador de batchs de datos
Imagen + label

Armar un generador que devuelva una imagen y su máscara,  en un tamaño específico de imagen, iterando sobre las imágenes de la carpeta images2D.

1. Tomar imagen : input_path -> image
2. Tomar máscara: input_path -> label
3. Generar output : (image,label)




```
mygenerator = segmentation_generator(images_2D,labels_2D,dim=(512,512))
img0,mask0 = next(mygenerator)
```



In [0]:
#Levantar una imagen y una mascara
sample_name = os.listdir(images_2D)[0]
print(sample_name)

In [0]:
from skimage.io import imread
from skimage.transform import resize

sample_img = imread(os.path.join(images_2D,sample_name),as_gray=True)
sample_img = resize(sample_img,(400,400))

plt.imshow(sample_img,cmap='gray')
print(sample_img.shape)

In [0]:
def segmentation_generator(imagespath,labelspath,dim=(512,512)):    
    sample_names = os.listdir(imagespath)
    for s in sample_names:
      img_path = os.path.join(....
      mask_path = os.path.join(....
      image = ....
      mask = .....
      yield (image,mask)

Ahora lo hacemos para un batch

In [0]:
def segmentation_generator(imagespath,labelspath,batch_size = 3,dim=(512,512)):    
    sample_names = os.listdir(imagespath)
    while True:
          # Seleccionar cuáles muestras usar en este batch
          batch_samplenames = np.random.choice(a = sample_names, 
                                         size = batch_size)
          batch_images = []
          batch_masks = [] 
          
          # Read in each input, perform preprocessing and get labels
          for input_path in batch_samplenames:
              img_path = os.path.join(.....
              mask_path = os.path.join (.....                      
              image = ....
              mask = ....

              image = np.expand_dims(image[:,:,0],axis=-1) 
              mask = np.expand_dims(mask[:,:,0],axis=-1) 
              batch_input += [ image ]
              batch_output += [ output ]
              
          # Return a tuple of (input,output) to feed the network
          batch_x = np.array( batch_input )
          batch_y = np.array( batch_output )
        
          yield( batch_x, batch_y )

## Generador para Keras

Feeding en tiempo real a tu modelo de Keras

Heredamos propiedades de la clase keras.utils.Sequence para ganar algunas funcionalidades copadas como multiprocessing.

La clase DataGenerator entonces tendrá que definir los siguientes métodos:


```
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(......)

    def __len__(....):
        'Denotes the number of batches per epoch'

    def __getitem__(......):
        'Generate one batch of data'

    def on_epoch_end(....):
        'Updates indexes after each epoch'

    def __data_generation(......):
        'Generates data containing batch_size samples' 
```

In [0]:
from keras.utils import Sequence

class ImagesMasksGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, images_dir, masks_dir, list_IDs=None, batch_size=8, dim=(512,512,1), shuffle=True):
        'Initialization'
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.dim = dim
        self.batch_size = batch_size
        self.shuffle = shuffle        
        self.images = [os.path.join(images_dir,file) for file in os.listdir(images_dir)]
        #print(self.images[0])
        self.list_IDs = list_IDs
        if list_IDs is None:
          self.list_IDs = list(range(len(self.images)))
        self.on_epoch_end()        

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        img, mask = self.__data_generation(list_IDs_temp)

        return img, mask

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        # Initialization
        imgs = np.empty((self.batch_size, *self.dim ))
        #print(imgs.shape)
        masks = np.empty((self.batch_size, *self.dim ))
        
        #imgs=np.empty(self.dim)
        #masks=np.empty(self.dim)
        
        # Generate data
        for i, ID in enumerate(list_IDs_temp): #list_IDs_temp = [0,3,67,90]
            img_path = self.images[ID]
            mask_path = os.path.join(self.masks_dir,os.path.basename(img_path))
            #print('Masks dir: ',self.masks_dir)
            #print('\nImg path: ', img_path)
            # Store sample
            x = resize(imread(img_path,as_gray=True),(self.dim[0],self.dim[1]))
            imgs[i,] = np.expand_dims(x,axis=-1)
            y = resize(imread(mask_path,as_gray=True),(self.dim[0],self.dim[1]))
            masks[i,] = np.expand_dims(y,axis=-1)
        #print('batchshape',imgs.shape)
        return imgs,masks

In [0]:
traingenerator = ImagesMasksGenerator(images_2D,labels_2D,batch_size=8)

In [0]:
next(traingenerator)

**Con validación**

In [0]:
paths = os.listdir(images_2D)
print(len(paths))
val_split = 5
input_idxs = np.random.permutation(len(paths))
split_idx = int(len(paths)*(1-val_split/100))
indexes_train = input_idxs[:split_idx]
indexes_val=input_idxs[split_idx:]


print("Cantidad de imagenes de entrenamiento: " + str(len(indexes_train)))
print("Cantidad de imagenes de validación: " + str(len(indexes_val)))


In [0]:
train_params = {'dim': (512, 512),
                'batch_size': 8,
                'shuffle': True,
                'list_IDs': indexes_train}

valid_params = {'dim': (512, 512),
                'batch_size': 3,
                'shuffle': True,
                'list_IDs': indexes_val}
                                 
                               
traingenerator = ImagesMasksGenerator(images_2D,labels_2D,**train_params)
valid_generator = ImagesMasksGenerator(images_2D,labels_2D,  **valid_params)

## DICE

Debemos hacer una función de costo propia, que permita comparar una imagen entera contra otra imagen entera (en lugar de un número contra otro número).

En segmentación se usa mucho la métrica del DICE: 

$$ Dice Coef = \frac{2|X \cap Y|}{|X|+|Y|} $$

In [0]:
!pip install medpy 
import medpy.metric.binary as metrics

In [0]:
example_img = loadnifty(example_img_path)[:,:,80]

example_mask = loadnifty(example_mask_path)[:,:,80]
displaced_example_mask = loadnifty(example_mask_path)[:,:,75]


example_perfect_dice = metrics.dc(example_mask,example_mask)
example_bad_dice = metrics.dc(displaced_example_mask,example_mask)

print('Ground truth con ground truth',example_perfect_dice)
print('Ground truth con desplazada',example_bad_dice)

plt.figure(figsize=(20,40))

plt.subplot(1,3,1)
plt.grid(False)
plt.title("Imagen de test: " + str(os.path.basename(example_img_path)))

plt.imshow(example_img, cmap='gray')

plt.subplot(1,3,2)
plt.grid(False)

plt.imshow(example_mask, cmap = 'viridis')
plt.subplot(1,3,3)
plt.grid(False)
plt.imshow(displaced_example_mask, cmap='viridis')
plt.imshow(example_mask, cmap='jet', alpha = 0.5)

 **Funciones de costo personalizadas**

Para esta segmentación necesitaremos implementar una función de costo propia. Veamos primero la estructura de una función de costo en Keras (casi idéntica en TensorFlow) con algunas de las funciones más usadas: 


In [0]:
def mean_squared_error(y_true, y_pred):    
    return K.mean(K.square(y_pred - y_true), axis=-1)

In [0]:
def categorical_crossentropy(y_true, y_pred):
    return K.categorical_crossentropy(y_true, y_pred)
  

  
def categorical_crossentropy(target, output, from_logits=False, axis=-1):
  """Categorical crossentropy between an output tensor and a target tensor.
  Arguments:
      target: A tensor of the same shape as `output`.
      output: A tensor resulting from a softmax
          (unless `from_logits` is True, in which
          case `output` is expected to be the logits).
      from_logits: Boolean, whether `output` is the
          result of a softmax, or is a tensor of logits.
      axis: Int specifying the channels axis. `axis=-1` corresponds to data
          format `channels_last', and `axis=1` corresponds to data format
          `channels_first`.
  Returns:
      Output tensor.
  Raises:
      ValueError: if `axis` is neither -1 nor one of the axes of `output`.
  """
  rank = len(output.shape)
  axis = axis % rank
  # Note: nn.softmax_cross_entropy_with_logits_v2
  # expects logits, Keras expects probabilities.
  if not from_logits:
    # scale preds so that the class probas of each sample sum to 1
    output = output / math_ops.reduce_sum(output, axis, True)
    # manual computation of crossentropy
    epsilon_ = _to_tensor(epsilon(), output.dtype.base_dtype)
    output = clip_ops.clip_by_value(output, epsilon_, 1. - epsilon_)
    return -math_ops.reduce_sum(target * math_ops.log(output), axis)
  else:
    return nn.softmax_cross_entropy_with_logits_v2(labels=target, logits=output)

**DICE EN TENSORFLOW**

In [0]:
import tensorflow as tf
def dice_coef(y_true, y_pred):
  numerator = 2 * tf.reduce_sum(y_true * y_pred)
  # some implementations don't square y_pred
  denominator = tf.reduce_sum(y_true + tf.square(y_pred))

  return numerator / (denominator + tf.keras.backend.epsilon())

In [0]:
def dice_loss(y_true,y_pred):
  return -dice_coef(y_true,y_pred)

## UNET: La red segmentadora

<img src="https://raw.githubusercontent.com/cmosquer/softmax/master/u-net-architecture2.png" alt="Drawing" style="width:40%;"/>


In [0]:
import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

 Implementar un U-Net que tenga:
 
 - Relu como función de activación
 - 32 feature maps a la salida del primer nivel de convolución.
 - 4 niveles de MaxPooling de 2x2
 - Que utilice 'he' como método de incialización de pesos: kernel_initializer = 'he_normal'

In [0]:
def unet(input_size = (512,512,1)):
    inputs = Input(input_size)    
    
    #PRIMERAS FLECHAS AZULES IZQUIERDA   
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = ...
    
    #PRIMERA FLECHA ROJA IZQUIERDA
    pool1 = MaxPooling2D(pool_size=(....

    #SEGUNDAS FLECHAS AZULES IZQUIERDA                               
    conv2 = ...
    conv2 = ...
                                    
    #SEGUNDA FLECHA ROJA IZQUIERDA                               
    pool2 = ...
    
    conv3 = ...
    conv3 = ...
    pool3 = ...
    
    conv4 = ...
    conv4 = ...
    pool4 = ...

    conv5 = ...
    conv5 = ...

    #PRIMERA FLECHA VERDE DERECHA (DESDE ABAJO)
    up1 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5))
                                    
    #PRIMERA FLECHA GRIS (DESDE ABAJO)                                
    merge1 = concatenate([conv4, up1], axis = 3)
    
    #PRIMERAS FLECHAS AZULES DERECHA (DESDE ABAJO)                                
    conv6 = ...
    conv6 = ....
    
    up2 = ...
    merge2 =...
    conv7 = ...
    conv7 = ...

    up3 = ...
    merge3 = ...
    conv8 = ...
    conv8 = ...

    up4 = ...
    merge4 = ...
    conv9 = ...
    conv9 = ...
    
    conv9=....
    out = Conv2D(1, 1, activation = 'sigmoid',name='output1')(conv9)
    
    model = Model(input = inputs, output = out)
    optimizer = Adam(lr=1e-5)
    # Compilar el modelo usando Adam como algoritmo de optimización, con learning rate = 1e-5
    model.compile(optimizer = optimizer, loss=dice_loss, metrics=[dice_coef])
    
    print("Model created")
    return model

##¡ENTRENAR!

In [0]:
model=unet()
model_checkpoint = ModelCheckpoint('unet_CMR.hdf5', monitor='loss',verbose=1, save_best_only=True)
history = model.fit_generator(traingenerator,steps_per_epoch=400, epochs=20,validation_data=validgenerator,callbacks=[model_checkpoint])


import pickle
with open('/content/gdrive/My Drive/Images/spleen_training/model_unet18-6.txt', mode='wb+') as f:
  pickle.dump({'model':model, 'history': history},f)

**Evaluar curvas de entrenamiento y validación**

In [0]:
import pickle
with open('/content/gdrive/My Drive/Images/spleen_training/model_unet18-6.txt', mode='rb') as f:
  dictresults=pickle.load(f)
  model= dictresults['model']
  hist=dictresults['hist']

## Predecir nuevas imágenes

Como utilizamos métricas y loss propias, debemos tener esas funciones definidas al cargar el modelo. Con el parámetro 'custom_objects' podemos indicar un diccionario de funciones propias.

In [0]:
from keras.models import load_model
from skimage.io import imread
from skimage.transform import resize


modelFile='/content/gdrive/My Drive/Images/spleen_training/unet2D_trained_example_model.hdf5'
model = load_model(modelFile, custom_objects={'dice_loss': dice_loss, 'dice_coef': dice_coef})

In [0]:
test_images_2D='/content/gdrive/My Drive/Images/spleen_training/unlabeled_images2D/'
testgenerator = ImagesMasksGenerator(test_images_2D,test_images_2D,batch_size=1,shuffle=False)
len(os.listdir(test_images_2D))

In [0]:
preds=model.predict_generator(testgenerator)
print(preds.shape)

In [0]:
f = plt.figure(figsize=(15,20))
# Visualizamos la imagen y el gt
plt.subplot(1,2,1)
plt.grid(False)
plt.imshow(imread(testgenerator.images[2],as_gray=True), cmap='gray')


plt.subplot(1,2,2)
plt.grid(False)
plt.imshow(preds[1,:,:,0])#, cmap='jet')

Para exportar una máscara binaria, necesitamos binarizar la máscara de salida. 
Definí un umbral y guardá esa máscara binaria para cada paciente de unlabeled_images2D. 

In [0]:
[...te toca implementar]

**Bonus**: aplicar un método de procesamiento de imágenes para refinar la segmentación. Por ejemplo:

- operaciones morfológicas. 
- buscar la etiqueta conexa de mayor tamaño y seleccionar solo esta. 
- etc

**Bonus 2**: aplicar la segmentación refinada a los volúmenes 3D en la carpeta unlabeled_images3D. Exportar una imagen tridimensional de salida con la máscara binaria tridimensional.

**Bonus 3**: entrenar un modelo que tome porciones de imagenes tridimensionales como entradas. Estos pequeños cubos, conocidos como pathces, es conveniente tenerlos hardcodeados de antemano. Es decir, el primer paso es recorrer todos los nifti y recortarlos  en "cubos", y guardar estos cubos individualmente (archivos .npy o .nii que tengan una matriz de 3 dimensiones). 